In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import single_useful_functions as uf
import pandas as pd
import warnings
from sklearn.metrics import  f1_score, accuracy_score

/isdata/winthergrp/zxv353/miniconda3/envs/deepenv/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


create dataset fasta for publication

full dataset

In [28]:
import re
locations = ['Cellwall', 'Extracellular', 'Cytoplasmic', 'CYtoplasmicMembrane', 'OuterMembrane', 'Periplasmic']

# read the fasta file
with open('./../data/fasta_graphpart_single_label.fasta', 'r') as f:
    data = f.readlines()

# loop through the data and modify the headers
for i in range(len(data)):
    if data[i].startswith('>'):
        header = data[i].strip()
        ID, rest = header.split("|")
        rest = rest.replace("label=", "")
        # Separate numbers and text
        numbers = ''.join(filter(str.isdigit, rest))
        kingdom = ''.join(filter(str.isalpha, rest))
        index = numbers.find("1")
        new_location = locations[int(index)]
        new_header = f"{ID}|{new_location}|{kingdom}"
        data[i] = new_header + "\n"
    else:
        data[i] = data[i].strip() + "\n"

with open('./../data/downloading_data/full_dataset.fasta', 'w') as f:
    f.writelines(data)

graphpart partition

In [54]:
with open('./../data/final_fasta_single_after_graph_part.fasta', 'r') as f:
    data = f.readlines()

partition = pd.read_csv("./../data/single_graphpart.csv")

# loop through the data and modify the headers
for i in range(len(data)):
    if data[i].startswith('>'):
        header = data[i].strip()
        ID, rest = header.split("|")
        # print(ID[1:])
        cluster = int(partition.loc[partition['AC'] == ID[1:], "cluster"].values)
        rest = rest.replace("label=", "")
        # Separate numbers and text
        numbers = ''.join(filter(str.isdigit, rest))
        kingdom = ''.join(filter(str.isalpha, rest))
        index = numbers.find("1")
        new_location = locations[int(index)]
        new_header = f"{ID}|{new_location}|{kingdom}|{cluster}"
        data[i] = new_header + "\n"
    else:
        data[i] = data[i].strip() + "\n"

with open('./../data/downloading_data/graphpart_set.fasta', 'w') as f:
    f.writelines(data)

benchmarking set

In [75]:
import pandas as pd

with open('./../data/final_fasta_single_after_graph_part.fasta', 'r') as f:
    data = f.readlines()

date = pd.read_csv(f"./../data/date_df.csv", index_col=0)
date['date'] = pd.to_datetime(date['date'])
filter_date = pd.to_datetime("2010-01-01")
date_2010 = date[date["date"] >= filter_date]

benchmarking = list(date_2010.index)

new_data = []  # new list to store the filtered data

# loop through the data and modify the headers
for i in range(0, len(data), 2):  # step by 2 because fasta file has 2 lines per record
    if data[i].startswith('>'):
        header = data[i].strip()
        ID, rest = header.split("|")
        if ID[1:] in benchmarking:
            # Separate numbers and text
            rest = rest.replace("label=", "")
            # Separate numbers and text
            numbers = ''.join(filter(str.isdigit, rest))
            kingdom = ''.join(filter(str.isalpha, rest))
            index = numbers.find("1")
            new_location = locations[int(index)]
            new_header = f"{ID}|{new_location}|{kingdom}\n"
            sequence = data[i+1].strip() + "\n"  # get the sequence line
            new_data.append(new_header)
            new_data.append(sequence)

with open('./../data/downloading_data/benchmarking_dataset.fasta', 'w') as f:
    f.writelines(new_data)